# 🔬 Strategy Development Template

Use this notebook to develop and backtest trading strategies before deploying to Freqtrade.

## Workflow:
1. Load historical data
2. Calculate indicators
3. Generate buy/sell signals
4. Backtest with VectorBT
5. Validate results
6. Convert to Freqtrade strategy

In [ ]:
import pandas as pd
import numpy as np
import vectorbt as vbt
from pathlib import Path

print("✅ Libraries loaded")

## 1️⃣ Load Data

In [ ]:
# Path to your data
data_path = Path("/home/jovyan/user_data/data")

# Load example pair
symbol = "BTC-USDT"
df = pd.read_json(data_path / f"{symbol}-5m.json")

print(f"Loaded {len(df)} candles for {symbol}")
df.head()

## 2️⃣ Calculate Indicators

In [ ]:
# Example: Simple Moving Average Crossover
df['sma_fast'] = df['close'].rolling(10).mean()
df['sma_slow'] = df['close'].rolling(50).mean()

df.dropna(inplace=True)
print(f"✅ Indicators calculated, {len(df)} candles remain")

## 3️⃣ Generate Signals

In [ ]:
# Buy when fast SMA crosses above slow SMA
entries = (df['sma_fast'] > df['sma_slow']) & (df['sma_fast'].shift(1) <= df['sma_slow'].shift(1))

# Sell when fast SMA crosses below slow SMA
exits = (df['sma_fast'] < df['sma_slow']) & (df['sma_fast'].shift(1) >= df['sma_slow'].shift(1))

print(f"🟢 Buy signals: {entries.sum()}")
print(f"🔴 Sell signals: {exits.sum()}")

## 4️⃣ Backtest with VectorBT

In [ ]:
# Run backtest
portfolio = vbt.Portfolio.from_signals(
    close=df['close'],
    entries=entries,
    exits=exits,
    init_cash=10000,
    fees=0.001,  # 0.1% trading fee
)

# Show stats
print(portfolio.stats())

## 5️⃣ Visualize Results

In [ ]:
# Plot equity curve
portfolio.plot().show()

# Plot trades on price chart
fig = vbt.plotting.Scatter(
    data=df['close'],
    trace_kwargs=dict(name='Price')
).update_xaxes_template(
    rangeslider_visible=False
).fig

fig.show()

## 6️⃣ Next Steps

1. Improve indicators and entry/exit rules
2. Test on multiple pairs and timeframes
3. Add risk management (stop loss, take profit)
4. Run walk-forward validation
5. Convert to Freqtrade strategy in `user_data/strategies/`

**Remember:** Past performance ≠ future results! Always test in dry-run first.